In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
import json

# Create Spark Session
# spark = SparkSession.builder \
#     .appName("MCC_Loader") \
#     .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
#     .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
#     .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123") \
#     .config("spark.hadoop.fs.s3a.path.style.access", "true") \
#     .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
#     .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
#     .config("spark.jars.packages", 
#             "org.apache.hadoop:hadoop-aws:3.3.4,"
#             "com.amazonaws:aws-java-sdk-bundle:1.12.262,"
#             "org.postgresql:postgresql:42.6.0") \
#     .getOrCreate()

spark = SparkSession.builder \
    .appName("MCC_Loader") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "1g") \
    .config("spark.executor.cores", "1") \
    .config("spark.cores.max", "3") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.jars.packages", 
            "org.apache.hadoop:hadoop-aws:3.3.4,"
            "com.amazonaws:aws-java-sdk-bundle:1.12.262,"
            "org.postgresql:postgresql:42.5.4") \
    .getOrCreate()

# Read JSON file from MinIO
#s3a://dev/users_data.csv
json_rdd = spark.sparkContext.textFile("s3a://dev/mcc_codes.json")
json_str = json_rdd.collect()[0]  # Get the entire JSON as string
mcc_dict = json.loads(json_str)

# Convert dict to list of tuples
mcc_data = [(code, desc) for code, desc in mcc_dict.items()]

# Define schema
schema = StructType([
    StructField("mcc_code", StringType(), False),
    StructField("description", StringType(), False)
])

# Create DataFrame
df = spark.createDataFrame(mcc_data, schema=schema)

# Show data
print("Data to be loaded:")
df.show(truncate=False)

# PostgreSQL properties
postgres_props = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}

# Load to PostgreSQL
df.write.jdbc(
    url="jdbc:postgresql://postgres:5432/gold_db",
    table="mcc_codes",
    mode="append",
    properties=postgres_props
)

print(f"Loaded {df.count()} records successfully!")
spark.stop()

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
import json

# Create Spark Session
# spark = SparkSession.builder \
#     .appName("MCC_Loader") \
#     .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
#     .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
#     .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123") \
#     .config("spark.hadoop.fs.s3a.path.style.access", "true") \
#     .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
#     .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
#     .config("spark.jars.packages", 
#             "org.apache.hadoop:hadoop-aws:3.3.4,"
#             "com.amazonaws:aws-java-sdk-bundle:1.12.262,"
#             "org.postgresql:postgresql:42.6.0") \
#     .getOrCreate()

spark = SparkSession.builder \
    .appName("MCC_Loader") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "1g") \
    .config("spark.executor.cores", "1") \
    .config("spark.cores.max", "3") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.jars.packages", 
            "org.apache.hadoop:hadoop-aws:3.3.4,"
            "com.amazonaws:aws-java-sdk-bundle:1.12.262,"
            "org.postgresql:postgresql:42.5.4") \
    .getOrCreate()

# Read JSON file from MinIO
#s3a://dev/users_data.csv
# df_mcc_codes = spark.read.option("multiline", "true").json("s3a://dev/mcc_codes.json")
# df_mcc_codes.show(10, truncate=False)

json_rdd = spark.sparkContext.textFile("s3a://dev/mcc_codes.json")
parsed_rdd = json_rdd.flatMap(lambda line: 
    [(k, v) for k, v in eval(line).items()]
)

schema = StructType([
    StructField("mcc_code", StringType(), True),
    StructField("description", StringType(), True)
])

df = spark.createDataFrame(parsed_rdd, schema)
df.show(truncate=False)

spark.stop()

Py4JJavaError: An error occurred while calling o147.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3) (172.18.0.6 executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_94/1694098372.py", line 48, in <lambda>
  File "<string>", line 1
    {
    ^
SyntaxError: '